In [43]:
#%pip install git+https://github.com/cair/tmu.git
#%pip install numpy==1.26.4

In [44]:
import numpy as np
import pickle

from tmu.models.autoencoder.autoencoder import TMAutoEncoder

In [45]:
def load_train_dataset(farm, event_id):
    X = np.loadtxt(f"./data_train/X_{farm}_{event_id}.txt", dtype=np.uint32)
    X = np.array(X).astype(np.uint32)
    return X

In [46]:
def log_results(epoch, precision, recall):
    with open("results.txt", "a") as f:
        f.write(f"{epoch},{precision},{recall}\n")

In [47]:
def save_model(tm: TMAutoEncoder, filename: str):
    a, d = tm.X_train, tm.encoded_X_train

    tm.X_train = None
    tm.encoded_X_train = None

    with open(filename, "wb") as f:
        pickle.dump(tm, f)

    tm.X_train = a
    tm.encoded_X_train = d

In [48]:
# TODO: Load X_train from a dataset
X_train = load_train_dataset("B", 34)

In [49]:
def train(args):
    tm = TMAutoEncoder(
        number_of_clauses=args["num_clauses"],
        T=args["T"],
        s=args["s"],
        output_active=args["output_active"],
        max_included_literals=args["max_included_literals"],
        accumulation=args["accumulation"],
        feature_negation=args["feature_negation"],
        platform=args["platform"],
        output_balancing=args["output_balancing"],
    )

    print(f"Starting training for {args['epochs']} epochs")

    for e in range(args["epochs"]):
        tm.fit(X_train, number_of_examples=args["number_of_examples"])

        save_model(tm, f"latest_{e}.pkl")
        print(f"Epoch: {e + 1}")


In [ ]:
number_of_features = 63 * 8
output_active = np.arange(number_of_features, dtype=np.uint32)

number_of_clauses = 100

args: dict = {
    "clause_weight_threshold": 1,
    "number_of_examples": 40000,
    "output_active": output_active,
    "accumulation": 1,
    "num_clauses": number_of_clauses,
    "T": int(number_of_clauses * 0.75 * 100),
    "s": 25,
    "epochs": 25,
    "platform": "CPU",
    "output_balancing": 0,
    "max_included_literals": number_of_features,
    "feature_negation": True,
}

result = train(args)

Starting training for 25 epochs
